In [1]:
import sys;sys.path.append('..')
from sqes_function import Config, DBPool
import numpy as np
import pandas as pd
from obspy import read_inventory
import requests
import json
import re
from tqdm import tqdm
from sqlalchemy import create_engine
from sqlalchemy.dialects import postgresql
from urllib.parse import quote

import warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

In [2]:
# Main data source

url = f'http://202.90.198.40/sismon-wrs/assets/sismon-slmon2/data/slmon.all.laststatus.json'
html = requests.get(url).content
json_data = json.loads(html)
json_data = json_data['features']

# list sta in json_data 
stations = []
for item in json_data:
    if 'properties' in item and 'sta' in item['properties']:
        stations.append(item['properties']['sta'])

In [3]:
# open db connection and read stations
db_config = Config.load_config(section="postgresql") 
db_pool = DBPool(**db_config) # type: ignore
encoded_password = quote(db_config['password'])
engine2 = create_engine(f"postgresql+psycopg2://{db_config['user']}:{encoded_password}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

# read stations from db
stations_db = pd.read_sql('select code,latitude,longitude from stations', con=engine2)

## UPDATE STATION METADATA

In [4]:
# see difference between json and db station
json_not_in_db = list(set(stations) - set(stations_db['code'].tolist()))
print(f"Stations in JSON not in DB: {json_not_in_db}")
db_not_in_json = list(set(stations_db['code'].tolist()) - set(stations))
print(f"Stations in DB not in JSON: {db_not_in_json}")

Stations in JSON not in DB: []
Stations in DB not in JSON: ['RC355', 'R02C2', 'R34E2', 'ERPI', 'SPSI', 'R799F', 'R7D14', 'R5F0D']


In [5]:
# insert new stations from json to db
if json_not_in_db:
    for sta in json_not_in_db:
        # find station in json_data
        station_data = next((item for item in json_data if item['properties']['sta'] == sta), None)
        if station_data:
            longitude = station_data['geometry'].get('coordinates', None)[0]
            latitude = station_data['geometry'].get('coordinates', None)[1]
            network = station_data['properties'].get('net', None)
            province = station_data['properties'].get('provin', None)
            location = station_data['properties'].get('location', None)
            year = None # no year data available in json
            upt = station_data['properties'].get('uptbmkg', None)
            balai = None
            digitizer_type = station_data['properties'].get('merkdgtz', None)
            match = re.search(r'(?:19|20)\d{2}-(.*)', digitizer_type) if digitizer_type else None
            communication_type = match.group(1) if match else None
            network_group = None
            
            if latitude is not None and longitude is not None:
                print(f"Inserting {sta} with lat: {latitude}, lon: {longitude}, network: {network}, province: {province}, location: {location}, year: {year}, upt: {upt}, balai: {balai}, digitizer_type: {digitizer_type}, communication_type: {communication_type}")
                # insert into db
                db_pool.execute(
                    "INSERT INTO stations (code, network, latitude, longitude, province, location, year, upt, balai, digitizer_type, communication_type, network_group) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                    (sta, network, latitude, longitude, province, location, year, upt, balai, digitizer_type, communication_type, network_group),
                    commit=True
                )
            else:
                print(f"Skipping {sta}, missing latitude or longitude.")
        else:
            print(f"Station {sta} not found in JSON data.")
    # update stations_db after insertion
    stations_db = pd.read_sql('select code,latitude,longitude from stations', con=engine2)
else:
    print("No new stations to insert from JSON.")

No new stations to insert from JSON.


In [30]:
# function see diff df1 that not in df2
def see_diff(df1, df2):
    diff_list = []
    if len(df1) != len(df2):
        print("differences:")
        df1_not_in_df2 = list(set(df1['code'].tolist()) - set(df2['code'].tolist()))
        for sta in df1_not_in_df2:
            if sta not in df2.code.tolist():
                diff_list.append(sta)
        print(diff_list)

# Sync stations data to other tables
stations_db = pd.read_sql('select code from stations', con=engine2)
stations_dominant_data_quality_db = pd.read_sql('select code from stations_dominant_data_quality', con=engine2)
stations_site_quality_db = pd.read_sql('select code from stations_site_quality', con=engine2)
stations_visit_db = pd.read_sql('select code from stations_visit', con=engine2)

print(f"** Stations in stations: {len(stations_db)} ; unique : {len(stations_db['code'].unique())}")
print(f"\n** Stations in stations_dominant_data_quality: {len(stations_dominant_data_quality_db)} ; unique : {len(stations_dominant_data_quality_db['code'].unique())}")
stations_not_in_dominant = list(set(stations_db['code'].tolist()) - set(stations_dominant_data_quality_db['code'].tolist()))
see_diff(stations_dominant_data_quality_db, stations_db)
# see_diff(stations_db, stations_dominant_data_quality_db)
print("stations_dominant_data_quality not in stations",len(stations_not_in_dominant),stations_not_in_dominant)

print(f"\n** Stations in stations_site_quality: {len(stations_site_quality_db)} ; unique : {len(stations_site_quality_db['code'].unique())}")
stations_not_in_site_quality = list(set(stations_db['code'].tolist()) - set(stations_site_quality_db['code'].tolist()))
see_diff(stations_site_quality_db, stations_db)
# see_diff(stations_db,stations_site_quality_db)
print("stations_site_quality not in stations",len(stations_not_in_site_quality),stations_not_in_site_quality)

print(f"\n** Stations in stations_visit: {len(stations_visit_db)} ; unique : {len(stations_visit_db['code'].unique())}")
stations_not_in_visit = list(set(stations_db['code'].tolist()) - set(stations_visit_db['code'].tolist()))
see_diff(stations_visit_db, stations_db)
# see_diff(stations_db, stations_visit_db)
print("stations_visit not in stations",len(stations_not_in_visit), stations_not_in_visit)


# Sync Process
print("-----------------------------------------------------------------")
# Sync stations_dominant_data_quality
if stations_not_in_dominant:
    for sta in stations_not_in_dominant:
        print(f"Inserting {sta} into stations_dominant_data_quality")
        db_pool.execute(
            "INSERT INTO stations_dominant_data_quality (code) VALUES (%s)",
            (sta,),
            commit=True
        )
print("Done inserting stations_dominant_data_quality")
print("-----------------------------------------------------------------")
# Sync stations_site_quality
if stations_not_in_site_quality:
    for sta in stations_not_in_site_quality:
        print(f"Inserting {sta} into stations_site_quality")
        db_pool.execute(
            "INSERT INTO stations_site_quality (code) VALUES (%s)",
            (sta,),
            commit=True
        )
print("Done inserting stations_site_quality")
print("-----------------------------------------------------------------")
# Sync stations_visit
if stations_not_in_visit:
    for sta in stations_not_in_visit:
        print(f"Inserting {sta} into stations_visit")
        db_pool.execute(
            "INSERT INTO stations_visit (code) VALUES (%s)",
            (sta,),
            commit=True
        )
print("Done inserting stations_visit")
print("-----------------------------------------------------------------")


** Stations in stations: 557 ; unique : 557

** Stations in stations_dominant_data_quality: 565 ; unique : 565
differences:
['SEJM', 'JMBI', 'SBJM', 'GGJI', 'BPJI', 'STPI', 'TNG', 'DWIKO']
stations_dominant_data_quality not in stations 0 []

** Stations in stations_site_quality: 557 ; unique : 557
stations_site_quality not in stations 0 []

** Stations in stations_visit: 557 ; unique : 557
stations_visit not in stations 0 []
-----------------------------------------------------------------
Done inserting stations_dominant_data_quality
-----------------------------------------------------------------
Done inserting stations_site_quality
-----------------------------------------------------------------
Done inserting stations_visit
-----------------------------------------------------------------


In [ ]:
# collect data from xml
rows_list = []
loop_obj = tqdm(stations_db['code'].tolist())
for code in loop_obj:
    try:
        loop_obj.set_description("Processing %s" % code)
        inv = read_inventory(f"https://geof.bmkg.go.id/fdsnws/station/1/query?station={code}&level=response&nodata=404")
        dict_xml = {
        'code': inv[0].stations[0].code,
        'latitude_xml': inv[0].stations[0].latitude,
        'longitude_xml' : inv[0].stations[0].longitude
        }
        rows_list.append(dict_xml)
    except Exception as e:
        print(f"Error processing station {code}: {e}")
stations_xml = pd.DataFrame(rows_list)

# merge
merge = stations_db.join(stations_xml.set_index('code'), on='code')
merge['latitude_diff'] = abs(merge.latitude - merge.latitude_xml)
merge['longitude_diff'] = abs(merge.latitude - merge.latitude_xml)

# check station lat and lon difference based on threshold
threshold = 0 #1/111
merge[(merge.latitude_diff > threshold) | (merge.longitude_diff > threshold)]

# update db
print("update stations latitude and longitude based on xml data")
sql="""UPDATE stations SET latitude=%s, longitude=%s WHERE code=%s"""
rowcount=0

for data in merge[(merge.latitude_diff > threshold) | (merge.longitude_diff > threshold)].iterrows():
    print("Processing %s" % data[1].code)
    # take data from station xml
    dict_xml = {
    'code': data[1].code,
    'latitude': data[1].latitude_xml,
    'longitude' : data[1].longitude_xml
    }
    # processing diff
    print(f"latitude_def : {data[1].latitude: >10} | latitude_xml : {dict_xml['latitude']: >10} | diff: {data[1].latitude_diff: >10}")
    print(f"longitude_def: {data[1].longitude: >10} | longitude_xml: {dict_xml['longitude']: >10} | diff: {data[1].longitude_diff: >10}")
    
    # update db
    db_pool.execute(
        sql,
        (dict_xml['latitude'], dict_xml['longitude'], dict_xml['code']),
        commit=True
    )   
    print("--------------------------------------------------------------------------------------------")
print("process finish")

## UPDATE STATIONS_SENSOR METADATA

In [ ]:
# update stations sensor data
stations_db = pd.read_sql('select code,latitude,longitude from stations', con=engine2)

# define unique columns for upsert
table_name = 'stations_sensor'
unique_cols = ['code','location','channel']

# setup upset method
def postgres_upsert_method(table, conn, keys, data_iter):
    insert_stmt = postgresql.insert(table.table).values(list(data_iter))
    on_conflict_stmt = insert_stmt.on_conflict_do_update(
        index_elements=unique_cols,
        set_={c.name: c for c in insert_stmt.excluded if c.name not in unique_cols}
    )
    conn.execute(on_conflict_stmt)

sensor_df = pd.DataFrame(columns=['code','location','channel','sensor'])
loop_obj = tqdm(stations_db['code'].tolist())
for station in loop_obj:
    try:
        loop_obj.set_description("Processing %s" % station)
        # get data
        url = f'http://202.90.198.40/sismon-wrs/web/detail_slmon2/{station}'
        html_ = requests.get(url).content
        df_list = pd.read_html(html_) # type: ignore
        
        # process data
        temp_df = df_list[0].copy()
        temp_df["Station/Channel"] = temp_df["Station/Channel"].str.split(" ")
        temp_df["channel"] = temp_df["Station/Channel"].apply(lambda x: x[1] if not x[1].isnumeric() else x[2])
        temp_df["location"] = temp_df["Station/Channel"].apply(lambda x: x[1] if x[1].isnumeric() else '')
        temp_df["sensor"] = temp_df["Sensor Type"]
        temp_df["code"] = temp_df["Station/Channel"].apply(lambda x: x[0])
        temp_df["Year"] = temp_df["Sensor Type"].apply(lambda x: x.split("-")[-1])
        temp_df = temp_df[["code","location","channel","sensor"]]

        # concanate data
        sensor_df = pd.concat([sensor_df,temp_df], ignore_index=True)

        # remove unavailable sensor data
        sensor_df = sensor_df[sensor_df.sensor != "xxx"]

        # clear
        del(temp_df)
    except Exception as e:
        print(f"Error processing station {station}: {e}")
        # if error, continue to next station
        continue

# push to database
print("--------------------------------------------------------------------------------------------")
print("Pushing data to Database")
sensor_df.to_sql('stations_sensor', 
                 con=engine2, 
                 if_exists='append', 
                 index=False,
                 method=postgres_upsert_method)

# details
print("--------------------------------------------------------------------------------------------")
print("Details:")
non_colocated = 0
non_colocated_list = []
colocated = 0
colocated_list = []
other = 0
other_list = []
for sensor in stations_db.code:
    tmp = sensor_df[sensor_df['code'] == sensor]
    if len(tmp) == 3:
        non_colocated+=1
        non_colocated_list.append(sensor)
    elif len(tmp) == 6:
        colocated+=1
        colocated_list.append(sensor)
    else:
        other+=1
        other_list.append(sensor)

print("non_colocated",non_colocated)
print(non_colocated_list)
print("colocated",colocated)
print(colocated_list)
print("other",other)
print(other_list)
print("--------------------------------------------------------------------------------------------")
print("process finish")

## OPTIONAL UPDATE METADATA WITH EXTERNAL FILE(S)

In [33]:
stations_db = pd.read_sql('select * from stations', con=engine2)
df = pd.read_csv("../../files/stasiun_seismik_prioritas_inatews_2025.csv")
print("stations_db columns:", stations_db.columns)
print("df columns:", df.columns)

stations_db columns: Index(['code', 'network', 'latitude', 'longitude', 'province', 'location',
       'year', 'upt', 'balai', 'digitizer_type', 'communication_type',
       'network_group'],
      dtype='object')
df columns: Index(['NO', 'NET', 'KODE', 'LOKASI', 'LINTANG', 'BUJUR', 'ELEVASI',
       'PROVINSI', 'UPT PENANGGUNG JAWAB', 'TAHUN INSTALASI SITE', 'JARINGAN',
       'PRIORITAS', 'ACCELEROMETER', 'DIGITIZER & KOMUNIKASI'],
      dtype='object')


In [34]:
# Checking
df_not_in_db = list(set(df['KODE'].tolist()) - set(stations_db['code'].tolist()))
print(f"Stations in df not in DB: {df_not_in_db}")
db_not_in_df = list(set(stations_db['code'].tolist()) - set(df['KODE'].tolist()))
print(f"Stations in db not in df: {db_not_in_df}")


Stations in df not in DB: ['STPI', 'KJPJI', 'JMBI']
Stations in db not in df: ['R5F0D', 'R799F', 'R34E2', 'R7D14', 'R02C2', 'RC355']


In [35]:
# Update metadata year and network_group
for sta in stations_db.code:
    if sta in df['KODE'].tolist():
        year = int(df[df['KODE'] == sta]['TAHUN INSTALASI SITE'].values[0])
        network_group = str(df[df['KODE'] == sta]['JARINGAN'].values[0])
        print(f"Updating {sta} with year: {year}, network_group: {network_group}")

        db_pool.execute(
            "UPDATE stations SET year=%s, network_group=%s WHERE code=%s",
            (year, network_group, sta),
            commit=True
        )
    else:
        print(f"Station {sta} not found in df, skipping update.")

Updating ALKI with year: 2012, network_group: LIBRA
Updating AAI with year: 2005, network_group: LIBRA
Updating ANAPI with year: 2023, network_group: IDRIP83
Updating APSI with year: 2007, network_group: LIBRA
Updating APSSI with year: 2023, network_group: IDRIP83
Updating ARKPI with year: 2023, network_group: IDRIP83
Updating ARMI with year: 2019, network_group: INA Type-A
Updating AAII with year: 2006, network_group: China
Updating ARSKI with year: 2023, network_group: IDRIP83
Updating ATKTI with year: 2023, network_group: IDRIP83
Updating ATNI with year: 2012, network_group: LIBRA
Updating BAJI with year: 2019, network_group: INA Type-B
Updating BBBCM with year: 2020, network_group: MR2020
Updating BAPJI with year: 2023, network_group: IDRIP83
Updating BASAI with year: 2023, network_group: ALOPTAMA 2023
Updating BATI with year: 2006, network_group: CTBTO
Updating BBCI with year: 2019, network_group: INA Type-A
Updating BBJI with year: 2016, network_group: GFZ
Updating BDBI with year

In [36]:
for i in stations_db.upt.unique():
    print(f"upt: {i} -> balai: {stations_db[stations_db.upt == i].balai.unique()}")

upt: Stageof-Kupang -> balai: [3]
upt: Stageof-Ambon -> balai: [4]
upt: Stageof-Sorong -> balai: [5]
upt: Stageof-Palu -> balai: [4]
upt: Stageof-Kepahiang -> balai: [2]
upt: Stageof-Angkasa -> balai: [5]
upt: Stageof-Maluku-Tenggara-Barat -> balai: [4]
upt: Stageof-Pasuruan -> balai: [3]
upt: Stageof-Gowa -> balai: [4]
upt: Stageof-Nganjuk -> balai: [3]
upt: Stageof-Aceh-Selatan -> balai: [1]
upt: Stageof-Bandung -> balai: [2]
upt: Stageof-Denpasar -> balai: [3]
upt: Stageof-Aceh-Besar -> balai: [1]
upt: Stageof-Kendari -> balai: [4]
upt: Stageof-Padang-Panjang -> balai: [1]
upt: Stageof-Ternate -> balai: [4]
upt: Stageof-Balikpapan -> balai: [3]
upt: Stageof-Banjarnegara -> balai: [2]
upt: Stageof-Alor -> balai: [3]
upt: Stageof-Jayapura -> balai: [5]
upt: Stageof-Tangerang -> balai: [2]
upt: Stageof-Nabire -> balai: [5]
upt: Stageof-Mataram -> balai: [3]
upt: Stageof-Deli-Serdang -> balai: [1]
upt: Stageof-Jayapura-Jayapura -> balai: [5]
upt: Stageof-Malang -> balai: [3]
upt: Stageo

## Update Dominant Data Quality

In [4]:
from datetime import datetime, timedelta

def generate_dates(start_date_iso, end_date_iso):
    # Parse the ISO 8601 date inputs
    start_date = datetime.fromisoformat(start_date_iso)
    end_date = datetime.fromisoformat(end_date_iso)
    
    # Check if start date is before end date
    if start_date > end_date:
        raise ValueError("Start date must be before end date")
    
    # Create a list to store the dates
    dates = []
    
    # Generate the list of dates in the ISO 8601 format
    current_date = start_date
    while current_date <= end_date:
        if current_date == end_date:
            date_str = current_date.strftime("%Y-%m-%dT23:59:59")
        else:
            date_str = current_date.strftime("%Y-%m-%dT00:00:00")
        dates.append(date_str)
        current_date += timedelta(days=1)
    
    return dates

In [5]:
stations_data_quality =  pd.read_sql(f'select code,result,date from stations_data_quality where date >= \'2025-01-01\' and date <= \'2025-12-31\'', engine2)

In [ ]:
dominant_value_dict = {}
for code in stations_data_quality.code.unique():
    value = stations_data_quality[stations_data_quality.code == code].value_counts(subset='result').index.values[0]
    dominant_value_dict[code]=value
df_dominant_quality = pd.DataFrame.from_dict(dominant_value_dict, orient='index',
                    columns=['dominant_data_quality'])
df_dominant_quality = df_dominant_quality.rename_axis('code').reset_index()
# rename dominant data quality condition
df_dominant_quality["dominant_data_quality"] = df_dominant_quality["dominant_data_quality"].replace({
    'Mati': 'No Data',
    'Buruk':'Poor',
    'Cukup Baik':'Fair',
    'Baik': 'Good'
})

In [10]:
sql="""UPDATE stations_dominant_data_quality SET dominant_data_quality=%s WHERE code=%s"""
for i,val in df_dominant_quality.iterrows():
    print(val.code,val.dominant_data_quality)
    with engine2.connect() as cur:
        cur.execute(sql,(val.dominant_data_quality,val.code,))

BAJI Good
CBJM Good
BESI Good
GENI Good
IHRSI Good
DNP Fair
BESM Good
BASAI Good
AAII Good
BWJI Fair
BGASI Good
GGJM Fair
BATI No Data
JAASI Good
EDFI Good
CIJI Good
ACJM Fair
JAGI Good
BYLI Good
CIJM Good
EDMPI Good
BBBCM Good
ALKI Good
BLCM Good
GPSM Good
CBJI Good
EGSI Good
BLJI Good
BKB Good
ANAPI Good
ESNI Fair
JCJI Good
GTJI Good
BBJI Good
FAKI Good
JHMI Good
BKNI No Data
GTOI Good
JMSI Fair
APSSI Fair
CSBJI Good
BBLSI Good
ARKPI Good
JPJI Good
BBSI Good
JSBFM Good
GARPI Good
CSJM Good
BDBI Good
JTJM Good
HINSI Good
GEBBI Good
BSMI Good
JWJM Good
KABKI Good
KARPI Good
BSSI Good
BDMUI Good
ARSKI Good
KASAI Good
BTSPI Good
IGBI Fair
KLI Good
DBJI Good
BUBSI Good
IHMI Good
KLJI Fair
KMNI Fair
LHSM Good
KMPI Good
MSHHI Good
ATNI Good
LISM Good
MMCI Good
NSBMM Good
KPJI Good
LLSM Good
MMRI Good
LMNI Good
MTJPI Good
OBMI No Data
KORPI Good


/tmp/ipykernel_158790/3644258014.py:5: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  cur.execute(sql,(val.dominant_data_quality,val.code,))


MASM Good
KRAI Fair
OMBFM Good
MTKI Fair
KRSI Good
KGCM Fair
MTMPI Good
KRJI Good
MBBI Good
PAASI Good
MTMTI Good
LASI Fair
KSI Good
KTMI Good
PABKI Good
MTSI Fair
KIMPI Good
LRTI Good
KUBSI Good
MBSM Good
KJCM Good
LSCM Good
KUKI Poor
MDSI Good
PAKI Fair
KKJM Good
DDSI Fair
BUMSI Good
BAKI Good
AAI Fair
BUSI Good
CCJM Good
BAPJI Good
GESM Good
IWPI Good
DOCM Good
DYPI No Data
CGJI Good
ACBM Good
BYJI Good
BGCI Fair
GHMI Good
BATPI Good
JAY No Data
BGCM Good
GMJI Good
ERPI No Data
CASI Good
JBJI Good
BKASI Good
BBCI Good
ALTI Good
JBJM Fair
GRJI Good
BBCM Good
ELMPI Good
CILJI Good
BLSM Good
BKJI Fair
CNJI Good
APSI Good
BMBNG Good
BBJM Good
FKMPM No Data
BOSM Good
BMSI Fair
FKSPI Fair
TOTSI Good
BPMJM Good
CSJI Good
BNDI Good
GUJM Fair
ARMI Fair
BSI Fair
GBJI Good
BNSI Fair
BDCM Good
ARPI Fair
CTJI Good
IATSM Good
KAKKI Good
GEJI Fair
BTJI Fair
CWJM Good
IBTI Good
BTCM Good
KMMI Good
ASTTI Good
KBBI Good
LHMI Good
BUKI Good
DBNFM Good
ATKTI Good
KBJM Fair
MLJI Good
MAKBI Good
MTAI Goo